In [1]:
import requests
import pandas as pd
# Import required info for access to the reddit api
import clientInfo as botInfo
import os

In [19]:
base_url = 'https://www.reddit.com/'
data = {'grant_type': 'password', 'username': botInfo.username, 'password': botInfo.password}
auth = requests.auth.HTTPBasicAuth(botInfo.client_id, botInfo.secret)
r = requests.post(base_url + 'api/v1/access_token',
                  data=data,
                  headers={'user-agent': 'CommentBot by u/ScraperBot-0'},
		          auth=auth)

d = r.json()

In [20]:
token = 'bearer ' + d['access_token']

base_url = 'https://oauth.reddit.com'

headers = {'Authorization': token, 'User-Agent': 'CommentBot by ScraperBot-0'}
response = requests.get(base_url + '/api/v1/me', headers=headers)

if response.status_code == 200:
    print("Connected")

Connected


In [21]:
subreddit = '/r/Art'
numOfPosts = 25
payload = {'limit': numOfPosts}
response = requests.get(base_url + subreddit, headers=headers, params=payload).json()

In [7]:
import json

posts = response["data"]["children"]
# number of posts
numPosts = len(posts)


def getResolution(datPost):
    resolution = datPost["preview"]["images"][0]["source"]
    return resolution["width"], resolution["height"]

# assign all post
def assignPostData(post: json, postInfo: dict):
    # get the url to the image/gif
    postInfo["url"].append(post["url_overridden_by_dest"])
    # get post title
    postInfo["title"].append(post["title"])
    # Write flag to show that it is an image or not
    postInfo["image"].append(True if not "gif" in post["url"] else False)
    # Find the width and the height of the image/gif
    width, height = getResolution(post)
    postInfo["height"].append(height)
    postInfo["width"].append(width)
    # Check to see if it's NSFW
    postInfo["NSFW"].append(True if "nsfw" in post["thumbnail"] else False)
    # Get number of upvotes and ratio of upvotes
    postInfo["ups"].append(post["ups"])
    postInfo["upRatio"].append(post["upvote_ratio"])
    

# Gets all posts from a json object including art work, returns a dictionary with all data of each post and the number of posts found
def getPostData(posts):
    # Stores information about the image
    postData = {
        "image":      [],
        "title":      [],
        "height":     [],
        "width":      [],
        "url":        [],
        'ups':        [],
        "upRatio":    [],
        "NSFW":       []
    }
    # Number of posts with art
    postCount = 0
    for i in range(len(posts)):
        post = posts[i]["data"]
        if "url_overridden_by_dest" in post and "preview" in post:
            # Increase the count of the posts
            postCount += 1
            # assign all data to the post
            assignPostData(post, postData)
    return postData, postCount


# Checks to see if the number of posts scraped is correct
def checkPostLength(postData: dict, numOfPosts: int):
    correctLen = True
    if len(postData["image"]) != numOfPosts:
        correctLen = False
    if len(postData["height"]) != numOfPosts:
        correctLen = False
    if len(postData["width"]) != numOfPosts:
        correctLen = False
    if len(postData["url"]) != numOfPosts:
        correctLen = False
    if len(postData["ups"]) != numOfPosts:
        correctLen = False
    if len(postData["upRatio"]) != numOfPosts:
        correctLen = False
    if len(postData["NSFW"]) != numOfPosts:
        correctLen = False
    return correctLen

In [22]:
posts

[{'kind': 't3',
  'data': {'approved_at_utc': None,
   'subreddit': 'Art',
   'selftext': "General Discussion threads are for casual chat; a place to ask for recommendations, lists, or creative feedback; to talk about materials, history, or techniques; and anything else that comes to mind.\n\nIf you're looking for information about a particular work of art, /r/WhatIsThisPainting is still the best resource.  /r/drawing , /r/painting , and /r/learnart may also be useful.   /r/ArtistLounge is also a good place for general discussion.  Please see our [list of art-related subs](https://www.reddit.com/r/Art/wiki/related) for more options.\n\n[Rule 8 still applies](https://www.reddit.com/r/Art/wiki/index#wiki_8._be_respectful_and_stay_on_topic) except that questions/complaints about r/Art and Reddit overall are allowed.\n\n***\n[Previous month's discussion](https://www.reddit.com/r/Art/comments/vrpq4v/general_discussion_thread_july_2022/)",
   'author_fullname': 't2_4hdw8',
   'saved': False,

In [8]:
postData, postCount = getPostData(posts)

In [15]:
def downloadPicture(url: str, filepath: str):
    picture = requests.get(postData["url"][i])
    # If a file already exists on the path delete the picture
    if(os.path.exists(filepath)):
            os.remove(filepath)
    with open(filepath, "wb") as f:
        f.write(picture.content)

filepath = "./wallpapers/wallpaper"

# Checks to see if the user would like a new wallpaper or change to a new one
# returns True to keep the wallpaper, False if not
def getUserInput():
    userChoice = input("Keep wallpaper y/n: ")
    if(userChoice == 'y' or userChoice == 'Y'):
        return True
    return False


for i in range(2, postCount):
    if(postData["image"][i] and not postData["NSFW"][i]):
        downloadPicture(postData["url"][i], filepath)
        # if getUserInput():
        #     break
        break